# Лабораторна робота №5. Семантична сегментація

**Виконала** студентка групи КІ-51мп Додонова Марія

**Мета:** Навчитись вирiшувати задачу семантичної сегментацiї зображень, дослiдити та реалiзувати архiтектури типу Encoder-Decoder, зокрема U-Net, а також ознайомитись iз сучасними foundation-моделями для сегментацiї.

## Порядок виконання

1. Обрати фреймворк для виконання роботи: `tensorflow` чи `pytorch`.

2. Обрати датасет для вирiшення задачi семантичної сегментацiї. Рекомендується почати з одного з наступних:
* [Oxford-IIIT Pet Dataset:](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet) Стандартний датасет для сегментацiї, що мiстить зображення домашнiх тварин 37 порiд.
* [Cityscapes Dataset:](https://www.cityscapes-dataset.com/) Зображення мiських пейзажiв. Бiльш складний та об’ємний.
* [МРТ-скани головного мозку:](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset) Приклад медичного датасету.

3. Реалiзувати модель U-Net власноруч, використовуючи стандартнi шари вашого фреймворку. Звернiть особливу увагу на реалiзацiю skip connections.

4. Навчити обрану модель на датасетi. Спробуйте використати специфiчну для сегментацiї функцiю втрат (наприклад, Dice Loss) та порiвняйте її результати зi стандартною Cross-Entropy.

5. Проаналiзувати графiки навчання. Оцiнити якiсть моделi за допомогою метрики mIoU на валiдацiйнiй вибiрцi. Обрати одну з архiтектур, реалiзованих у лабораторнiй роботi №2 або №3 (наприклад, вашу власну просту CNN, VGG або ResNet), яка буде слугувати базовою моделлю для експериментiв.

6. Вiзуалiзувати результати роботи моделi: для кiлькох зображень з тестової вибiрки показати оригiнальне зображення, справжню маску сегментацiї та маску, згенеровану вашою моделлю.

* **Завдання на додатковi бали:** Дослiдити одну з сучасних foundation моделей для сегментацiї. Оберiть один з варiантiв:

  * **Segment Anything Model (SAM) / SAM 2:** Використовуючи [офiцiйну демо-версiю](https://segment-anything.com/) або [готовий код](https://github.com/facebookresearch/segment-anything), застосуйте модель до кiлькох власних зображень. Опишiть, як працює prompt-based segmentation (сегментацiя за пiдказками: точки, рамки).
  * **Zero-shot Segmentation:** Дослiдiть моделi, що можуть сегментувати об’єкти за текстовим описом без попереднього навчання на конкретних класах (наприклад, на основi CLIP). Реалiзуйте приклад zero-shot сегментацiї, використовуючи одну з [доступних моделей на Hugging Face](https://huggingface.co/models?pipeline_tag=zero-shot-image-segmentation&sort=trending).

## Виконання роботи

### 1. Обрати фреймворк для виконання роботи: `tensorflow` чи `pytorch`.

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, random_split

from src.train.trainer import Trainer
from src.losses.diceloss import DiceLoss
from src.plot_tools import plot_history, imshow
from src.models.unet import UNet

DATA_PATH = "data"
DEVICE = "cuda" if torch.cuda.is_available() else "cpu"
NUM_EPOCHS = 10
BATCH_SIZE = 32
LEARNING_RATE = 0.001

print(f"Using device: {DEVICE}")

### 2. Обрати датасет для вирiшення задачi семантичної сегментацiї. Рекомендується почати з одного з наступних:
* [Oxford-IIIT Pet Dataset:](https://www.tensorflow.org/datasets/catalog/oxford_iiit_pet) Стандартний датасет для сегментацiї, що мiстить зображення домашнiх тварин 37 порiд.
* [Cityscapes Dataset:](https://www.cityscapes-dataset.com/) Зображення мiських пейзажiв. Бiльш складний та об’ємний.
* [МРТ-скани головного мозку:](https://www.kaggle.com/datasets/masoudnickparvar/brain-tumor-mri-dataset) Приклад медичного датасету.

In [ ]:
class TargetToLong(nn.Module):
    def forward(self, target: torch.Tensor) -> torch.Tensor:
        target = (target * 255 - 1).to(torch.long)
        return target


transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor()
])

target_transform = transforms.Compose([
    transforms.Resize((128, 128)),
    transforms.ToTensor(),
    TargetToLong()
])

# Load train split
train_dataset = datasets.OxfordIIITPet(
    root=DATA_PATH,
    split="trainval",
    target_types="segmentation",
    download=True,
    transform=transform,
    target_transform=target_transform
)

# Load test split
test_dataset = datasets.OxfordIIITPet(
    root=DATA_PATH,
    split="test",
    target_types="segmentation",
    download=True,
    transform=transform,
    target_transform=target_transform
)

# Split train into train and validation
train_dataset, val_dataset = random_split(train_dataset, [0.8, 0.2])

# Create DataLoaders
train_loader = DataLoader(train_dataset, batch_size=BATCH_SIZE, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=BATCH_SIZE)
test_loader = DataLoader(test_dataset, batch_size=BATCH_SIZE)

### 3. Реалiзувати модель U-Net власноруч, використовуючи стандартнi шари вашого фреймворку. Звернiть особливу увагу на реалiзацiю skip connections.

### 4. Навчити обрану модель на датасетi. Спробуйте використати специфiчну для сегментацiї функцiю втрат (наприклад, Dice Loss) та порiвняйте її результати зi стандартною Cross-Entropy.

In [ ]:
print("Training U-Net with DiceLoss:")
dl_criterion = DiceLoss()
dl_unet = UNet(in_channels=3)
dl_optimizer = optim.Adam(dl_unet.parameters(), lr=LEARNING_RATE)
dl_trainer = Trainer(dl_unet, dl_optimizer, dl_criterion, device=DEVICE)
dl_history = dl_trainer.train(NUM_EPOCHS, train_loader, val_loader)
print("Test results:", end=" ")
dl_trainer.test(test_loader)

print("\nTraining U-Net with CrossEntropyLoss:")
ce_criterion = nn.CrossEntropyLoss()
ce_unet = UNet(in_channels=3)
ce_optimizer = optim.Adam(ce_unet.parameters(), lr=LEARNING_RATE)
ce_trainer = Trainer(ce_unet, ce_optimizer, ce_criterion, device=DEVICE)
ce_history = ce_trainer.train(NUM_EPOCHS, train_loader, val_loader)
print("Test results:", end=" ")
ce_trainer.test(test_loader)

print("\nFinished Training!")

### 5. Проаналiзувати графiки навчання. Оцiнити якiсть моделi за допомогою метрики mIoU на валiдацiйнiй вибiрцi. Обрати одну з архiтектур, реалiзованих у лабораторнiй роботi №2 або №3 (наприклад, вашу власну просту CNN, VGG або ResNet), яка буде слугувати базовою моделлю для експериментiв.

In [ ]:
print("U-Net (DiceLoss) Curves")
plot_history(dl_history)

print("U-Net (DiceLoss) Curves")
plot_history(ce_history)

### 6. Вiзуалiзувати результати роботи моделi: для кiлькох зображень з тестової вибiрки показати оригiнальне зображення, справжню маску сегментацiї та маску, згенеровану вашою моделлю.

In [ ]:
for images, labels in test_loader:
    dl_unet.eval()
    ce_unet.eval()
    images = images.to(DEVICE)
    with torch.no_grad():
        dl_out = dl_unet(images)
        ce_out = ce_unet(images)
    
    images = images.to("cpu").permute(0, 2, 3, 1).numpy()
    labels = labels.permute(0, 2, 3, 1).numpy()
    dl_pred = torch.argmax(dl_out, dim=1).to("cpu").unsqueeze(-1).numpy()
    ce_pred = torch.argmax(ce_out, dim=1).to("cpu").unsqueeze(-1).numpy()
    for i in range(3):
        imshow(images[i], "Input image")
        imshow(labels[i], "Label")
        imshow(dl_pred[i], "Prediction (DiceLoss)")
        imshow(ce_pred[i], "Prediction (CrossEntropyLoss)")
    break

### * **Завдання на додатковi бали:** Дослiдити одну з сучасних foundation моделей для сегментацiї. Оберiть один з варiантiв:

  * **Segment Anything Model (SAM) / SAM 2:** Використовуючи [офiцiйну демо-версiю](https://segment-anything.com/) або [готовий код](https://github.com/facebookresearch/segment-anything), застосуйте модель до кiлькох власних зображень. Опишiть, як працює prompt-based segmentation (сегментацiя за пiдказками: точки, рамки).

  * **Zero-shot Segmentation:** Дослiдiть моделi, що можуть сегментувати об’єкти за текстовим описом без попереднього навчання на конкретних класах (наприклад, на основi CLIP). Реалiзуйте приклад zero-shot сегментацiї, використовуючи одну з [доступних моделей на Hugging Face](https://huggingface.co/models?pipeline_tag=zero-shot-image-segmentation&sort=trending).

## Висновок